In [1]:
!pip install tldextract


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
import tldextract
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [7]:
df = pd.read_csv('Phishing_Websites_Data.csv')
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [8]:
# Check target column
print(df.columns)

# Assuming 'Result' is the target
X = df.drop('Result', axis=1)
y = df['Result']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))


Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')
✅ Accuracy: 0.9669832654907282


In [9]:
def extract_features_from_url(url):
    features = []

    # Feature 1: URL length
    features.append(len(url))

    # Feature 2: IP address in URL
    ip_pattern = re.compile(r'(([0-9]{1,3}\.){3}[0-9]{1,3})')
    features.append(1 if re.search(ip_pattern, url) else -1)

    # Feature 3: '@' symbol
    features.append(1 if '@' in url else -1)

    # Feature 4: Number of subdomains
    ext = tldextract.extract(url)
    features.append(ext.subdomain.count('.'))

    # Feature 5: HTTPS check
    features.append(1 if url.startswith("https") else -1)

    # Feature 6: Extra slashes after protocol
    features.append(1 if url.find('//', 8) != -1 else -1)

    # Feature 7: Hyphen in domain
    features.append(1 if '-' in ext.domain else -1)

    # Feature 8: Length of domain
    features.append(len(ext.domain))

    return np.array(features).reshape(1, -1)


In [10]:
# Sample training URLs
sample_urls = [
    "https://www.google.com",
    "http://192.168.0.1/login.php",
    "https://secure.amazon.com",
    "http://paypal-login.verify.com@phish.ru"
]
sample_labels = [1, -1, 1, -1]  # 1 = Legit, -1 = Phishing

# Convert URLs to features
X_simple = np.array([extract_features_from_url(u).flatten() for u in sample_urls])
y_simple = np.array(sample_labels)

# Train a mini model
url_model = RandomForestClassifier(n_estimators=100, random_state=42)
url_model.fit(X_simple, y_simple)


RandomForestClassifier(random_state=42)

In [15]:
# Quick Test (No Waits or Loops)
test_url = "https://secure.amazon.com"  # change to any test case

features = extract_features_from_url(test_url)
prediction = url_model.predict(features)
print("🔎 Result:", "Phishing 🚨" if prediction[0] == -1 else "Legitimate ✅")


🔎 Result: Legitimate ✅
